In [1]:
import json
with open('summary-meta') as f:
    line = f.readline()

category_topic_list = list(json.loads(line))
#for content in category_topic_list:
 #   print(content)
print(len(category_topic_list))    


1766


In [2]:

with open('summary-docs') as f:
    contentLines = f.readlines()

contentLines = [ line.strip() for line in contentLines]


In [3]:
len(contentLines)

1766

In [4]:

contentLines = [ line.lower() for line in contentLines]

In [5]:
###TBC preprocess 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


def preprocessing(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #remove stop words
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]
    
    ##lower case
    tokens = [token.lower() for token in tokens]
    
    ##lemma
    lmtzr = WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)
    return (text)
    
print(preprocessing('she is good better running apples ran'))

contentLines = map(lambda x: preprocessing(x) ,contentLines)
    

good better running apple ran


In [6]:
###sklearn CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1,2),min_df=2,stop_words='english',norm='l2') ##用三过于稀疏了 只有0.65准确率在验证ji
###best: 用ngrame_range=(1,2) 不做任何特征处理时候validation accuracy 0.780 training accuracy 0.998 特征维度 (1766, 77971)
ngram_content = cv.fit_transform(contentLines).toarray()
print(ngram_content.shape)

(1766, 16520)


In [42]:
##转化成category id list 用dict存name to id mapping
category_text_list = [line['Category'] for line in category_topic_list]
category_text_set = set(category_text_list)
categort2id_dict = dict(zip(category_text_set,range(len(category_text_set))))
category_id_list = map(lambda x: categort2id_dict[x] ,category_text_list)
len(category_id_list)
id2c_dict = dict(zip(range(len(category_text_set)),category_text_set))

###看一下分类数量
arr_appear = dict((a, category_text_list.count(a)) for a in category_text_list);
print(arr_appear)
class_weight = dict((a, category_id_list.count(a)) for a in category_id_list);
print(class_weight)
print(id2c_dict)

{u'taxi': 32, u'overseas news': 1, u'public transport industry': 2, u'forum': 66, u'smrt corporate': 533, u'competitors': 1, u'smrt buses': 368, u'unknown': 13, u'smrt taxis': 77, u'smrt trains': 673}
{0: 32, 1: 1, 2: 2, 3: 66, 4: 533, 5: 1, 6: 368, 7: 13, 8: 77, 9: 673}
{0: u'taxi', 1: u'overseas news', 2: u'public transport industry', 3: u'forum', 4: u'smrt corporate', 5: u'competitors', 6: u'smrt buses', 7: u'unknown', 8: u'smrt taxis', 9: u'smrt trains'}


In [2]:
# encoding=utf-8
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("我好開心啊")  # 搜索引擎模式
print(", ".join(seg_list))


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
我, 好, 開心, 啊


In [8]:
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score
import numpy as np
import pandas as pd
from pandas import DataFrame

/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  Deprec

In [9]:

###Optional: elect feature
from sklearn.feature_selection import RFE
print "feature selection"    
##for train, test in cv:
svc = linear_model.LogisticRegression(C=5)
selector = RFE(svc, 10000, step=0.1)
features = selector.fit_transform(ngram_content, category_id_list)
print "feature selection finshed"

feature selection
feature selection finshed


In [14]:
##train using SVM: TB Tuned
 
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(ngram_content,np.array(category_id_list),test_size=0.1)



In [15]:

 
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(ngram_content,np.array(category_id_list),test_size=0.1)
svm = SVC(kernel=str('linear'))
print("start","train")
svm.fit(X_train, y_train)
print("finish","train")

('start', 'train')
('finish', 'train')


In [16]:
test_result = svm.predict(X_test)
print(len(test_result))
results =  (test_result == y_test) 
score = [1 for i in results if i]
accuracy = float(sum(score))/float(len(results))
print('validation accuracy %.3f' %accuracy )
print('training accuracy %.3f' %svm.score(X_train,y_train ) )

177
validation accuracy 0.774
training accuracy 0.972


In [48]:

from sklearn.metrics import precision_score, recall_score, f1_score
for i in range(10):
    p = precision_score(y_test[test_result==i], test_result[test_result==i],average='weighted')
    print(" p is {0} {1} {2}".format(p,id2c_dict[i],sum(y_test == i)))
r = recall_score(y_test, test_result,average='weighted')
f1score = f1_score(y_test, test_result,average='weighted')
 

print(r)
print(f1score)


 p is 0 taxi 2
 p is 0 overseas news 0
 p is 0 public transport industry 0
 p is 1.0 forum 11
 p is 0.5625 smrt corporate 59
 p is 0 competitors 0
 p is 0.758584807492 smrt buses 29
 p is 0 unknown 2
 p is 0.25 smrt taxis 8
 p is 0.567139155617 smrt trains 66
0.762711864407
0.733358732832


In [ ]:
##各种model来一遍

In [24]:
####ensemble
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC

lr = linear_model.LogisticRegression(C=5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)    
svm = SVC(kernel=str('linear'))
rf = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=0.48979591836734693, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False, n_estimators = 100)
eclf1 = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('svm', svm)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
print("end")

/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

end


In [25]:
test_result = eclf1.predict(X_test)
print(len(test_result))
results =  (test_result == y_test) 
score = [1 for i in results if i]
accuracy = float(sum(score))/float(len(results))
print('validation accuracy %.3f' %accuracy ) ###提高0.02
print('training accuracy %.3f' %eclf1.score(X_train,y_train ) )

177
validation accuracy 0.763
training accuracy 0.967


In [ ]:

print "logistic回归"    
for train, test in cv:
    lr = linear_model.LogisticRegression(C=5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False).fit(ngram_content[train], np.array(category_id_list)[train])    
    #linear_model.LogisticRegression(C=5,class_weight='balanced').fit(ngram_content[train], np.array(category_id_list)[train])    
   
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        svc.score(ngram_content[train], np.array(category_id_list)[train]), svc.score(ngram_content[test], np.array(category_id_list)[test])))


In [12]:
# 总得切分一下数据咯（训练集和测试集）
df_train_data = DataFrame()
cv = cross_validation.ShuffleSplit(len(ngram_content), n_iter=20, test_size=0.1,
    random_state=0)

# 各种模型来一圈


In [20]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.svm import SVC

print "ensemble bagging"
lr = linear_model.LogisticRegression(C=5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)    
svm = SVC(kernel=str('linear'))
rf = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=0.48979591836734693, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False, n_estimators = 100)

for train, test in cv:
    eclf1 = clfb = BaggingClassifier(base_estimator= svm)
                         
    eclf1 = eclf1.fit(ngram_content[train], np.array(category_id_list)[train])
    
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        eclf1.score(ngram_content[train], np.array(category_id_list)[train]), eclf1.score(ngram_content[test], np.array(category_id_list)[test])))


ensemble bagging
train score: 0.339, test score: 0.311

train score: 0.041, test score: 0.017



KeyboardInterrupt: 

In [13]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC

print "ensemble"
lr = linear_model.LogisticRegression(C=5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)    
svm = SVC(kernel=str('linear'))
rf = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=0.48979591836734693, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False, n_estimators = 100)

for train, test in cv:
    eclf1 = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('svm', svm)], voting='hard')
    eclf1 = eclf1.fit(ngram_content[train], np.array(category_id_list)[train])
    
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        eclf1.score(ngram_content[train], np.array(category_id_list)[train]), eclf1.score(ngram_content[test], np.array(category_id_list)[test])))


ensemble


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.971, test score: 0.808



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.969, test score: 0.819



...

train score: 0.963, test score: 0.808



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.969, test score: 0.802



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.968, test score: 0.791



In [ ]:
# 总得切分一下数据咯（训练集和测试集）
df_train_data = DataFrame()
cv = cross_validation.ShuffleSplit(len(ngram_content), n_iter=20, test_size=0.1,
    random_state=0)

# 各种模型来一圈


In [58]:
###gridsearch using logistic regression

tuned_parameters = [{'C':[1,5,10],'penalty':["l1",'l2']}]   
    
scores = ['r2']

for score in scores:
    
    print score
    
    clf = GridSearchCV(linear_model.LogisticRegression(class_weight='balanced'), tuned_parameters, cv=5)
    clf.fit(ngram_content, np.array(category_id_list))

    print("别！喝！咖！啡！了！最佳参数找到了亲！！：")
    print(clf.best_score_)
    print ""
    #best_estimator_ returns the best estimator chosen by the search
    print(clf.best_estimator_)
    print ""
    print("得分分别是:")
    print ""
    #grid_scores_的返回值:
    #    * a dict of parameter settings
    #    * the mean score over the cross-validation folds 
    #    * the list of scores for each fold
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print ""


r2


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


别！喝！咖！啡！了！最佳参数找到了亲！！：
0.814835787089

LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

得分分别是:

0.712 (+/-0.009) for {'penalty': 'l1', 'C': 1}
0.800 (+/-0.005) for {'penalty': 'l2', 'C': 1}
0.772 (+/-0.004) for {'penalty': 'l1', 'C': 5}
0.811 (+/-0.006) for {'penalty': 'l2', 'C': 5}
0.775 (+/-0.007) for {'penalty': 'l1', 'C': 10}
0.815 (+/-0.006) for {'penalty': 'l2', 'C': 10}



In [11]:
   
print "随机森林回归/Random Forest(n_estimators = 100)"    
for train, test in cv:    
    
    svc = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=0.48979591836734693, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False, n_estimators = 100).fit(ngram_content[train], np.array(category_id_list)[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(svc.score(ngram_content[train], np.array(category_id_list)[train]), svc.score(ngram_content[test], np.array(category_id_list)[test])))


随机森林回归/Random Forest(n_estimators = 100)


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.904, test score: 0.757



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.906, test score: 0.774



...

train score: 0.905, test score: 0.751



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.901, test score: 0.718



/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Use

train score: 0.902, test score: 0.734



In [ ]:
###gridsearch using rf

##tuned_parameters = [{'n_estimators':[100],'max_depth':[5,10,50,None],'min_impurity_split':np.linspace(0,1,50),
##                    }]  

tuned_parameters=[{'criterion':['entropy'],'min_impurity_split':np.linspace(0,1,50)},
                 {'criterion':['gini'],'min_impurity_split':np.linspace(0,0.5,50)},
                  {'max_depth':range(2,10)},
                  {'min_samples_split':range(2,30,2)}]
    
scores = ['r2']

for score in scores:
    
    print score
    
    clf = GridSearchCV(RandomForestClassifier(n_estimators=10,class_weight='balanced'), tuned_parameters, cv=5)
    clf.fit(ngram_content, np.array(category_id_list))

    print("别！喝！咖！啡！了！最佳参数找到了亲！！：")
    print ""
    #best_estimator_ returns the best estimator chosen by the search
    print(clf.best_estimator_)
    print ""
    print("得分分别是:")
    print ""
    #grid_scores_的返回值:
    #    * a dict of parameter settings
    #    * the mean score over the cross-validation folds 
    #    * the list of scores for each fold
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print ""


r2


/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/fay/anaconda/lib/python2.7/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.


别！喝！咖！啡！了！最佳参数找到了亲！！：

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=0.48979591836734693, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

得分分别是:

0.730 (+/-0.008) for {'min_impurity_split': 0.0, 'criterion': 'entropy'}
0.738 (+/-0.010) for {'min_impurity_split': 0.020408163265306121, 'criterion': 'entropy'}
0.712 (+/-0.003) for {'min_impurity_split': 0.040816326530612242, 'criterion': 'entropy'}
0.732 (+/-0.008) for {'min_impurity_split': 0.061224489795918366, 'criterion': 'entropy'}
0.717 (+/-0.010) for {'min_impurity_split': 0.081632653061224483, 'criterion': 'entropy'}
0.717 (+/-0.009) for {'min_impurity_split': 0.1020408163265306, 'criterion': 'entropy'}
0.725

In [29]:
print "支持向量回归/SVC linear 最好适用于多维度小数据m》n 否则用高斯rbf "

for train, test in cv:
    svc = svm.SVC(kernel=str('linear')).fit(ngram_content[train], np.array(category_id_list)[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(svc.score(ngram_content[train], np.array(category_id_list)[train]), svc.score(ngram_content[test], np.array(category_id_list)[test])))
##

支持向量回归/SVC linear 最好适用于多维度小数据m》n 否则用高斯rbf 
train score: 0.969, test score: 0.819

train score: 0.971, test score: 0.831

train score: 0.968, test score: 0.831

train score: 0.969, test score: 0.819

train score: 0.967, test score: 0.814

train score: 0.969, test score: 0.808

train score: 0.973, test score: 0.802

train score: 0.973, test score: 0.802

train score: 0.970, test score: 0.831

train score: 0.972, test score: 0.802

train score: 0.970, test score: 0.808

train score: 0.972, test score: 0.802

train score: 0.970, test score: 0.774

train score: 0.969, test score: 0.831

train score: 0.972, test score: 0.876

train score: 0.972, test score: 0.785

train score: 0.969, test score: 0.831

train score: 0.966, test score: 0.814

train score: 0.971, test score: 0.785

train score: 0.972, test score: 0.791



In [27]:
###gridsearch using rf

tuned_parameters = [{'C':[1,5,10,20]}]   
    
scores = ['r2']

for score in scores:
    
    print score
    
    clf = GridSearchCV(svm.SVC(kernel=str('linear'),class_weight='balanced'), tuned_parameters, cv=5)
    clf.fit(ngram_content, np.array(category_id_list))

    print("别！喝！咖！啡！了！最佳参数找到了亲！！：")
    print ""
    #best_estimator_ returns the best estimator chosen by the search
    print(clf.best_estimator_)
    print ""
    print("得分分别是:")
    print ""
    #grid_scores_的返回值:
    #    * a dict of parameter settings
    #    * the mean score over the cross-validation folds 
    #    * the list of scores for each fold
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print ""


r2
别！喝！咖！啡！了！最佳参数找到了亲！！：

SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

得分分别是:

0.809 (+/-0.004) for {'C': 1}
0.808 (+/-0.009) for {'C': 5}
0.801 (+/-0.009) for {'C': 10}
0.804 (+/-0.010) for {'C': 20}



In [9]:
print(len(contentLines))

1766


In [10]:
whole_words = []
print(contentLines[1].split(' '))
#for line in contentLines:
#    contentLineWords = line.strip()
    

[u'route', u'smrt', u'bus', u'service', u'75', u'167', u'amended', u'sunday', u'(', u'may', u'1', u')', u',', u'anderson', u'bridge', u'direction', u'st', u'andrew', u"'s", u'road', u'closed', u'bus', u'heavy', u'vehicle', u'.', u'route', u'service', u'amended', u'ply', u'esplanade', u'drive', u'instead', u'.']


In [89]:
import re
temp = "?!@forum writer mr han jiajing ( `` shorten interval time peak-hour train '' ; monday , mar 2 ) . argued fare increase given approval raised , one would expected train service improved . opined train service seem deteriorating instead seems attempt improve even basic . noticed cabin packed brim train arrive five six minute apart , even platform get overcrowded , result , travelling older train east-west line uncomfortable , especially air-conditioning working cabin . opined overcrowded condition safety concern well . free travel peak hour introduced , smrt assured commuter train would arrive two three minute apart , argued scenario materialise . mr han contended although land transport authority promised improvement , would take another two year happen . meantime , least smrt could ensure train arrival time improved air-conditioning older train maintained comfortable level . concluded train fare increase would justifiable improvement ."
temp = temp.decode("utf8")
string = re.sub("[\.,\(\);:\?!@]\'","",temp)
print string


?!@forum writer mr han jiajing ( `` shorten interval time peak-hour train '' ; monday , mar 2 ) . argued fare increase given approval raised , one would expected train service improved . opined train service seem deteriorating instead seems attempt improve even basic . noticed cabin packed brim train arrive five six minute apart , even platform get overcrowded , result , travelling older train east-west line uncomfortable , especially air-conditioning working cabin . opined overcrowded condition safety concern well . free travel peak hour introduced , smrt assured commuter train would arrive two three minute apart , argued scenario materialise . mr han contended although land transport authority promised improvement , would take another two year happen . meantime , least smrt could ensure train arrival time improved air-conditioning older train maintained comfortable level . concluded train fare increase would justifiable improvement .


In [95]:
##print(contentLines[3])
for i in range(len(contentLines)):
    contentLines[i] = re.sub("[\.,\(\);:\?!@\'\`]","",contentLines[i])
print(contentLines[90])

reader  ni xu gang metta  shared disappointment massive mrt breakdown july 7 remains grateful smrt staff worked overnight agonising period  disturbed fact even though smrt staff tried best restore system  endure harsh remark numerous member public  highlight many complained missing dinner family member  forget smrt staff also affected  many recalled back work crisis 


In [1]:
print(contentLines[3])
for line in contentLines:
    whole_words.extend(line.split())
    print(len(line.split()))
print(len(whole_words))

NameError: name 'contentLines' is not defined

In [97]:
whole_words = []
for line in contentLines:
    whole_words.extend(line.split())
    print(len(line.split()))
print(len(whole_words))

word_dict = set(whole_words)
print(len(word_dict))
print(contentLines[2:10])

0
28
95
122
31
68
65
74
46
35
27
6
63
89
84
48
7
6
146
128
223
39
6
84
93
49
17
9
67
7
6
72
42
99
48
48
30
123
55
28
209
35
61
46
47
117
176
48
9
47
12
52
18
37
9
56
75
53
48
46
13
15
23
57
44
29
92
36
80
21
67
26
46
46
43
40
43
90
129
36
63
47
49
72
59
13
8
78
30
38
53
43
21
52
190
49
62
9
6
61
47
57
146
144
18
51
100
11
6
59
128
40
23
56
33
15
40
32
32
66
50
8
54
64
74
86
38
5
10
212
7
59
12
36
92
34
39
7
45
42
42
233
62
39
107
51
22
61
79
17
35
41
69
17
6
63
33
58
75
8
44
42
63
7
56
147
67
67
38
154
50
8
36
9
52
52
119
37
55
39
41
49
43
26
56
23
57
218
52
35
8
65
44
45
8
45
55
54
32
175
144
55
45
178
58
199
67
112
93
21
18
44
6
88
41
56
49
35
49
42
39
56
47
40
88
65
83
36
47
51
55
51
7
11
4
119
67
81
13
118
12
38
7
74
63
38
112
2
55
91
92
41
90
53
32
97
29
93
36
72
50
68
35
64
63
5
7
39
14
50
9
55
31
55
7
54
81
19
13
63
63
51
51
63
55
46
54
71
86
57
60
5
39
71
40
33
10
55
60
56
33
64
42
129
98
21
37
46
9
87
42
59
84
50
37
97
62
24
55
36
40
7
34
48
117
81
50
6
70
45
43
31
69
93
40
9


In [98]:
word_2_int = { k:v for k, v in zip(word_dict,range(1,len(word_dict)+1))  }

In [99]:
print(len(word_dict))

9776


In [100]:
print(len(range(len(word_dict))))

9776


In [101]:
print(len(word_2_int))

9776


In [102]:
print(min(word_2_int.values()))

1


In [103]:
print(word_2_int['car'])

3079


In [104]:
print(word_2_int)

{u'stock': 6856, u'raining': 2, u'yellow': 3, u'factory': 8153, u'four': 5, u'railing': 6, u'localized': 7, u'looking': 8, u'four-month-old': 9, u'electricity': 10, u'aileen': 6533, u'unanswered': 12, u'servicesgovernment': 13, u'networkmost': 86, u'620pm': 15, u'1223pm': 16, u'sinking': 17, u'commercialisation': 8844, u'regional': 19, u'lijuan': 20, u'bringing': 21, u'prize': 22, u'wooden': 23, u'wednesday': 25, u'29-year-old': 26, u'persisted': 27, u'commented': 29, u'lahowyan': 30, u'specially': 31, u'ngee': 2676, u'270': 33, u'consists': 4817, u'second': 35, u'valiant': 36, u'ventilationour': 37, u'278': 38, u'melvin': 39, u'theory': 8160, u'dangled': 41, u'heinimann': 42, u'addedseparately': 43, u'contributed': 312, u'pre-mature': 45, u'fossil': 46, u'resilient': 47, u'notified': 7503, u'increasing': 48, u'specialist': 49, u'misjudged': 50, u'hero': 51, u'reporter': 52, u'donalds': 53, u'qunyu': 54, u'jasmine': 55, u'here': 56, u'reported': 57, u'ching': 58, u'china': 59, u'chink'

In [105]:
def word2Int(wd):
    try:
        return(word_2_int[wd])
    except KeyError:
        return 0

print(word2Int(u'nuk'), word2Int(u'train'))

(0, 6828)


In [106]:
print(word2Int(u'nuk'), word2Int(u'train'))

(0, 6828)


In [107]:
def categoryInt2Array(idx,dim):
    category_list = [0]*dim
    category_list[idx] = 1
    return category_list

In [108]:
print(categoryInt2Array(2,10))

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [109]:
train_list = []
for line in contentLines:
    line_split = line.split()
    train_list.append(map(lambda x:word2Int(x),line_split))

In [110]:
print(len(train_list))
print(train_list[1])

1766
[4401, 1202, 7271, 238, 8874, 5223, 9063, 5576, 5587, 8078, 7878, 7584, 8372, 744, 8375, 981, 6029, 9530, 7271, 3763, 4280, 4401, 238, 9063, 2902, 4820, 913, 9308]


In [131]:
maxlen = 100
from keras.preprocessing import sequence
train_list = sequence.pad_sequences(train_list, maxlen=maxlen) ###padding
print(train_list[10:20])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 7860 6828 2564 3369  238  589  720 1449 1505 1572 3780
  5385 1458 6333 9409 6017 6828 3795  238 5150 1853 1723 2247 8312 8414
  1202 6828]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0

In [132]:
print(len(category_id_list))
print(len(train_list))
print(category_id_list[0])
train_y_list = map(lambda x: categoryInt2Array(x,10), category_id_list)
print(len(train_y_list))
print(train_y_list[1])

1766
1766
4
1766
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [133]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
model = Sequential()
model.add(Embedding(len(word_dict)+1, 64, input_length = maxlen))
model.add(LSTM(256,return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
#model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")

#model = Sequential()
#model.add(Dense(512, input_shape=(200,), activation='tanh'))

#model.add(Dropout(0.5))
#model.add(Dense(10))
#model.add(Activation('softmax'))
#model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', class_mode="categorical")


In [134]:
help(Embedding)

Help on class Embedding in module keras.layers.embeddings:

class Embedding(keras.engine.topology.Layer)
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
 |  
 |  This layer can only be used as the first layer in a model.
 |  
 |  # Example
 |  
 |  ```python
 |    model = Sequential()
 |    model.add(Embedding(1000, 64, input_length=10))
 |    # the model will take as input an integer matrix of size (batch, input_length).
 |    # the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
 |    # now model.output_shape == (None, 10, 64), where None is the batch dimension.
 |  
 |    input_array = np.random.randint(1000, size=(32, 10))
 |  
 |    model.compile('rmsprop', 'mse')
 |    output_array = model.predict(input_array)
 |    assert output_array.shape == (32, 10, 64)
 |  ```
 |  
 |  # Arguments
 |    input_dim: int > 0. Size of the vocabulary,
 |        i.e. maximum int

In [135]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', class_mode="categorical")
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 64)           625728    
_________________________________________________________________
lstm_13 (LSTM)               (None, 100, 256)          328704    
_________________________________________________________________
dropout_13 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
Total para

In [136]:
X_train,X_test,y_train,y_test=train_test_split(train_list,np.array(train_y_list),test_size=0.1)                      

print(len(X_train),len(y_train))
print(len(X_test),len(y_test))

(1589, 1589)
(177, 177)


In [137]:
import numpy as np

model.fit(np.array(X_train),np.array(y_train), validation_data = (X_test,y_test),epochs = 20, batch_size=32)

Train on 1589 samples, validate on 177 samples
Epoch 1/20
1589/1589 [==============================] - 77s - loss: 1.6279 - acc: 0.3354 - val_loss: 1.5079 - val_acc: 0.1921
Epoch 2/20
1589/1589 [==============================] - 70s - loss: 1.2667 - acc: 0.4959 - val_loss: 0.9375 - val_acc: 0.7119
Epoch 3/20
1589/1589 [==============================] - 73s - loss: 0.9274 - acc: 0.6916 - val_loss: 0.8223 - val_acc: 0.7232
Epoch 4/20
1589/1589 [==============================] - 71s - loss: 0.7073 - acc: 0.7741 - val_loss: 0.8825 - val_acc: 0.7514
Epoch 5/20
1589/1589 [==============================] - 82s - loss: 0.5028 - acc: 0.8238 - val_loss: 0.7603 - val_acc: 0.7627
Epoch 6/20
1589/1589 [==============================] - 72s - loss: 0.3813 - acc: 0.8798 - val_loss: 0.7302 - val_acc: 0.7627
Epoch 7/20
1589/1589 [==============================] - 71s - loss: 0.2572 - acc: 0.9194 - val_loss: 0.8263 - val_acc: 0.7627
Epoch 8/20
1589/1589 [==============================] - 70s - loss: 0.1

In [122]:
score = model.evaluate(X_test,y_test)

177/177 [==============================] - 1s     


In [123]:
help(model.evaluate)
print(score)

Help on method evaluate in module keras.models:

evaluate(self, x, y, batch_size=32, verbose=1, sample_weight=None) method of keras.models.Sequential instance
    Computes the loss on some input data, batch by batch.
    
    # Arguments
        x: input data, as a Numpy array or list of Numpy arrays
            (if the model has multiple inputs).
        y: labels, as a Numpy array.
        batch_size: integer. Number of samples per gradient update.
        verbose: verbosity mode, 0 or 1.
        sample_weight: sample weights, as a Numpy array.
    
    # Returns
        Scalar test loss (if the model has no metrics)
        or list of scalars (if the model computes other metrics).
        The attribute `model.metrics_names` will give you
        the display labels for the scalar outputs.
    
    # Raises
        RuntimeError: if the model was never compiled.

[2.3179928205781062, 0.7175141263142818]
